# TP4 — A/B Testing RetailRocket



In [3]:
import pandas as pd
import numpy as np
from math import erf, sqrt
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 4)

DATA_PATH = "../Data/events.csv"
RANDOM_SEED = 42


In [4]:
raw_df = pd.read_csv(DATA_PATH)
raw_df.head()


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [5]:
print("Colonnes :", raw_df.columns.tolist())
print("\nTypes d'événements :")
print(raw_df['event'].value_counts())
print("\nProportion des événements :")
print(raw_df['event'].value_counts(normalize=True))
print("\nVolumétrie totale :", len(raw_df))


Colonnes : ['timestamp', 'visitorid', 'event', 'itemid', 'transactionid']

Types d'événements :
event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

Proportion des événements :
event
view           0.9667
addtocart      0.0252
transaction    0.0081
Name: proportion, dtype: float64

Volumétrie totale : 2756101


In [6]:
events_to_keep = ["view", "addtocart"]
filtered_df = raw_df[raw_df["event"].isin(events_to_keep)].copy()
filtered_df.reset_index(drop=True, inplace=True)

missing_summary = filtered_df.isna().sum()
print("Valeurs manquantes par colonne:\n", missing_summary)
print("\nNombre d'événements par type:\n", filtered_df['event'].value_counts())
print("\nVisiteurs uniques (filtrés) :", filtered_df['visitorid'].nunique())


Valeurs manquantes par colonne:
 timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

Nombre d'événements par type:
 event
view         2664312
addtocart      69332
Name: count, dtype: int64

Visiteurs uniques (filtrés) : 1407500


In [7]:
rng = np.random.default_rng(RANDOM_SEED)
visitor_groups = (
    filtered_df[["visitorid"]]
    .drop_duplicates()
    .assign(group=lambda df: rng.choice(["A", "B"], size=len(df)))
)

ab_df = filtered_df.merge(visitor_groups, on="visitorid", how="left")

group_distribution = visitor_groups['group'].value_counts(normalize=True)
print("Répartition des groupes (unique visitors):\n", group_distribution)
print("\nVérification sur les événements :\n", ab_df['group'].value_counts(normalize=True))


Répartition des groupes (unique visitors):
 group
A    0.5002
B    0.4998
Name: proportion, dtype: float64

Vérification sur les événements :
 group
B    0.5014
A    0.4986
Name: proportion, dtype: float64


In [8]:
kpi_table = (
    ab_df.groupby(["group", "event"])
    .size()
    .unstack(fill_value=0)
    .rename(columns={"view": "views", "addtocart": "add_to_cart"})
)

kpi_table["add_to_cart_rate"] = kpi_table["add_to_cart"] / kpi_table["views"]
kpi_table


event,add_to_cart,views,add_to_cart_rate
group,,,
A,34329,1328655,0.0258
B,35003,1335657,0.0262


In [9]:
rate_a = kpi_table.loc["A", "add_to_cart_rate"]
rate_b = kpi_table.loc["B", "add_to_cart_rate"]
rate_diff = rate_b - rate_a

print(f"Taux A : {rate_a:.4%}")
print(f"Taux B : {rate_b:.4%}")
print(f"Différence (B - A) : {rate_diff:.2%}")


Taux A : 2.5837%
Taux B : 2.6207%
Différence (B - A) : 0.04%


### Formules utilisées

Les calculs du test de proportion reposent sur les définitions suivantes :

- Proportion combinée (pooled proportion) :\
  $\displaystyle p = \frac{c_A + c_B}{n_A + n_B}$
- Statistique de test :\
  $\displaystyle Z = \frac{p_B - p_A}{ \sqrt{ p(1-p) \left( \frac{1}{n_A} + \frac{1}{n_B} \right) } }$
- p-value bilatérale :\
  $\displaystyle \text{p-value} = 2 \times \left( 1 - \Phi(|Z|) \right )$

avec :
- $c_A, c_B$ : nombre d'add-to-cart observés dans chaque groupe
- $n_A, n_B$ : nombre de vues
- $p_A, p_B$ : taux d'ajout au panier de chaque groupe
- $\Phi$ : fonction de répartition de la loi normale centrée réduite


In [10]:
views_a = kpi_table.loc["A", "views"]
views_b = kpi_table.loc["B", "views"]
adds_a = kpi_table.loc["A", "add_to_cart"]
adds_b = kpi_table.loc["B", "add_to_cart"]

p_a = rate_a
p_b = rate_b
p_pool = (adds_a + adds_b) / (views_a + views_b)

def norm_cdf(x: float) -> float:
    return 0.5 * (1 + erf(x / sqrt(2)))

z_score = (p_b - p_a) / np.sqrt(p_pool * (1 - p_pool) * (1 / views_a + 1 / views_b))
p_value = 2 * (1 - norm_cdf(abs(z_score)))

print(f"Z-score : {z_score:.3f}")
print(f"p-value bilatérale : {p_value:.4f}")
print("Conclusion (alpha=0.05) :", "Différence significative" if p_value < 0.05 else "Pas de différence significative")


Z-score : 1.893
p-value bilatérale : 0.0584
Conclusion (alpha=0.05) : Pas de différence significative


## Analyse business

- **Comparaison des taux** : le groupe B présente un add-to-cart rate supérieur (différence calculée ci-dessus).
- **Significativité statistique** : la p-value issue du test de proportions indique si l'écart est statistiquement détectable au seuil de 5%. Si la p-value < 0.05, on peut attribuer l'amélioration à la variante B avec un risque d'erreur de type I inférieur à 5%.
- **Décision** :
  - Si l'effet est significatif et positif, recommander le déploiement de B.
  - Si l'effet n'est pas significatif, conserver A et/ou envisager un test C avec de nouvelles hypothèses.
- **Risque** : documenter explicitement la probabilité d'erreur (p-value) et vérifier la puissance du test si besoin (taille d'échantillon suffisante).

> Adapter la recommandation finale en fonction des résultats numériques retournés ci-dessus.


### Résultats numériques du test

- Proportion combinée : $p = \frac{34329 + 35003}{1328655 + 1335657} = 0.0260$ (2.60%).
- Statistique de test : $Z = 1.893$.
- p-value bilatérale : $0.0584$.
- Décision au seuil $\alpha = 0.05$ : **on ne rejette pas $H_0$** (l'écart B vs A n'est pas statistiquement significatif malgré un différentiel positif de ~0.04 point).

> Interprétation : la variante B montre un léger gain sur le taux d'ajout au panier, mais le test ne fournit pas assez d'évidence pour conclure à une amélioration réelle. Il est recommandé de poursuivre la collecte de données ou d'itérer sur une nouvelle variante avant un déploiement global.


### Recommandation business finale

Le test montre un léger gain du groupe B (+0,04 point de taux d'ajout au panier) mais la p-value de 0,0584 ne permet pas de rejeter $H_0$ au seuil de 5%. Dans ces conditions :

- Conserver **design A** comme version de référence tant que la significativité n'est pas atteinte.
- Planifier une itération **variante C** avec des améliorations UX plus marquées (mise en avant du CTA, personnalisation) et relancer un test avec une taille d'échantillon suffisante pour augmenter la puissance.
- Surveiller les KPI business (add-to-cart rate, chiffre d'affaires) si un nouveau test est lancé avant tout déploiement global.

Conclusion : pas de déploiement généralisé du design B à ce stade; privilégier une nouvelle expérimentation mieux dimensionnée avant de changer de design en production.


## Rapport détaillé 

### 1. Contexte et objectif
- **Contexte** : analyse du dataset RetailRocket `events.csv` (événements `view`, `addtocart`, `transaction`).
- **Objectif** : évaluer, via un test A/B simulé, si la variante B de la page produit améliore le taux d’ajout au panier par rapport à la variante A.
- **KPI principal** : add-to-cart rate = nombre d’`addtocart` / nombre de `view`.

### 2. Préparation et nettoyage des données
- Chargement de **2 756 101** événements avec les colonnes : `timestamp`, `visitorid`, `event`, `itemid`, `transactionid`.
- Filtrage sur les événements pertinents : conservation de `view` et `addtocart` uniquement.
- Résultat après filtrage : **2 664 312 vues** et **69 332 add-to-cart**, pour **1 407 500 visiteurs uniques**.
- Qualité des données : aucune valeur manquante sur les colonnes utilisées (`timestamp`, `visitorid`, `event`, `itemid`), `transactionid` est manquant mais non utilisé.

### 3. Simulation de la randomisation A/B
- Randomisation **par utilisateur** : chaque `visitorid` est assigné une seule fois à A ou B (50/50), avec une seed fixe (`RANDOM_SEED = 42`) pour la reproductibilité.
- La table d’assignation est fusionnée avec les événements filtrés pour créer la colonne `group`.
- Répartition des groupes :
  - Sur les visiteurs : ≈ **50,02 %** en A et **49,98 %** en B.
  - Sur les événements : ≈ **49,86 %** en A et **50,14 %** en B.
- Conclusion : la randomisation est bien équilibrée et respecte le protocole d’un A/B test.

### 4. KPI : add-to-cart rate par groupe
À partir du tableau `kpi_table` :

- **Groupe A** :
  - `views` = 1 328 655
  - `addtocart` = 34 329
  - **add-to-cart rate** \(p_A\) = **2,5837 %**

- **Groupe B** :
  - `views` = 1 335 657
  - `addtocart` = 35 003
  - **add-to-cart rate** \(p_B\) = **2,6207 %**

- **Différence observée** :
  - \(p_B - p_A \approx 0{,}00037\), soit **+0,037 point de pourcentage**, environ **+1,4 %** d’amélioration relative pour B.

### 5. Test statistique de comparaison de proportions
- Hypothèses :
  - \(H_0 : p_A = p_B\) (pas d’effet de la variante B).
  - \(H_1 : p_A \neq p_B\) (la variante B modifie le taux d’ajout au panier).

- Proportion combinée (pooled) :
  - \(p = \frac{c_A + c_B}{n_A + n_B} = 0{,}0260\) (2,60 %).

- Statistique de test :
  - \(Z = 1{,}893\).

- p-value bilatérale :
  - **p-value = 0,0584**.

- Décision au seuil \(\alpha = 0{,}05\) :
  - p-value > 0,05 ⇒ **on ne rejette pas \(H_0\)** : l’écart entre A et B n’est pas statistiquement significatif.

### 6. Analyse business
- Le groupe B montre un **léger avantage** en taux d’ajout au panier, mais cet effet est **faible** et non confirmé statistiquement.
- Déployer B comme “gagnant” introduirait un **risque d’erreur de type I** (croire à une amélioration qui n’existe pas réellement).
- Pour détecter de si petits écarts avec une bonne puissance, il faudrait soit prolonger l’expérience, soit concevoir une variante plus différenciante.

### 7. Recommandation produit finale
- **Design à conserver** : maintenir **le design A** en production, faute de preuve solide que B soit meilleur.
- **Suite proposée** :
  - Concevoir une **variante C** avec des changements UX plus marqués (CTA plus visible, éléments de réassurance, recommandations produits…) et définir à l’avance la différence minimale détectable souhaitée.
  - Lancer un nouveau test A/B dimensionné en conséquence (taille d’échantillon suffisante, puissance statistique ≥ 80 %).
  - Suivre non seulement le **add-to-cart rate**, mais aussi des indicateurs business comme le **revenu par visiteur** avant tout déploiement global.


